# Experiment with Pytorch
to create endocder decoder ocr model with cnn and lstm

In [1]:
# Setup path in .env file
import os
from dotenv import load_dotenv
import cv2
import numpy as np
import torch

# Get absolut path to proeject root
load_dotenv()
project_root_dir = os.path.dirname(os.path.abspath("./"))
print("Project root dir:", project_root_dir)

data_dir = os.getenv("PUBTABNET_CROPPED_DATA_DIR")
absolute_dir = project_root_dir + data_dir
print("Absolute path:", absolute_dir)

Project root dir: /Users/leonremke/Documents/GIT_REPOS/UNI/neural_networks_seminar
Absolute path: /Users/leonremke/Documents/GIT_REPOS/UNI/neural_networks_seminar/PubTabNet_cropped


In [2]:
image_dir_train = f"{absolute_dir}/train"
image_dir_val = f"{absolute_dir}/val"
label_file_train = f"{absolute_dir}/PubTabNet_2.0.0_train_separated.json"
label_file_val = f"{absolute_dir}/PubTabNet_2.0.0_val_separated.json"

In [3]:
import sys
sys.path.append('../')
from torch.utils.data import DataLoader
label_file_small = f"{absolute_dir}/subset_small.json"


In [4]:
# # Function to generate labels
# import json
# def generate_labels(json_data):
#     labels = []
#     for key, value in json_data.items():
#         cells = value["html"]["cells"]
#         for i, cell in enumerate(cells):
#             bbox = cell.get('bbox')
#             tokens = cell.get('tokens')
#             if bbox is None:
#                 continue
#             label = key.replace(".png", f"_bbox_{bbox[0]}_{bbox[1]}_{bbox[2]}_{bbox[3]}.png")
#             label = {
#                 "filename": key.replace(".png", f"_bbox_{bbox[0]}_{bbox[1]}_{bbox[2]}_{bbox[3]}.png"),
#                 "split": value["split"],
#                 "imgid": value["imgid"],
#                 "tokens": tokens,
#                 "bbox": bbox,
#             }
#             labels.append(label)
#     return labels

# # Generate labels
# with open(label_file_train, 'r') as f:
#         labels = json.load(f)
#         result = generate_labels(labels)

# # Specify the output file name
# output_file_name = 'PubTabNet_2.0.0_train_separated.json'

# # Write the generated labels to a new JSON file
# with open(output_file_name, 'w') as output_file:
#     json.dump(result, output_file, indent=4)

# print(f"Generated labels have been written to {output_file_name}")

In [5]:
# Example usage:
from torchvision import transforms

from custom_ocr_cnn_lstm.dataset import OCRDataset
from custom_ocr_cnn_lstm.dataset_cropped import OCRDatasetCropped

transform = transforms.Compose([transforms.Resize((224, 224)),
                                transforms.ToTensor()])

train_dataset = OCRDatasetCropped(label_file_train, image_dir_train, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
val_dataset = OCRDatasetCropped(label_file_val, image_dir_val, transform=transform)
val_loader = DataLoader(val_dataset, batch_size=10, shuffle=True)

# Accessing a sample
sample_image, sample_label = train_dataset[0]
print(sample_image.shape)  # Check the shape of the image tensor
print(sample_label)         # Check the label dictionary for the 

torch.Size([3, 224, 224])
['<b>', 'S', 'p', 'e', 'c', 'i', 'e', 's', '</b>']


In [6]:
import json
def create_alphabet_file(alphabet, file_path):
    with open(file_path, 'w') as f:
        f.write("START\n")
        for char in alphabet:
            f.write(char + "\n")
        f.write("END\n")

# Replace these with your actual ground truth labels
train_labels = train_dataset.labels
val_labels = train_dataset.labels
# Concat both dicts
labels = train_labels + val_labels

alph_labels = []
for label in labels:
    label_tokens = label.get('tokens')
    label = " ".join(" ".join(tokens) for tokens in label_tokens)
    label = label.replace("< b >", " ").replace("< / b >", " ")
    alph_labels.append(label)
print("Ground truth labels loaded: ", len(alph_labels), alph_labels)

unique_chars = set()
unique_radicals = set()

# Loop through ground truth labels to extract unique characters and radicals
for label in alph_labels:
    for char in label:
        unique_chars.add(char)
        # You might need to extract radicals from each character here if using decomposition

# Define the file paths for character and radical alphabets
char_alphabet_file_path = f"{absolute_dir}/character_alphabet.txt"
radical_alphabet_file_path = f"{absolute_dir}/radical_alphabet.txt"

# Create character alphabet file
create_alphabet_file(unique_chars, char_alphabet_file_path)
print(f"Character alphabet file created at: {char_alphabet_file_path}")

# Create radical alphabet file (if needed)
# create_alphabet_file(unique_radicals, radical_alphabet_file_path)
# print(f"Radical alphabet file created at: {radical_alphabet_file_path}")

Ground truth labels loaded:  13690 ['  S p e c i e s  ', '  A n a j a ́ s  ', '  P o r t e l  ', '  S S B V  ', '  W i l d  ', '  R u r a l  ', '  U r b a n  ', '  W i l d  ', '  R u r a l  ', '  U r b a n  ', '  W i l d  ', '  R u r a l  ', '  U r b a n  ', '  T o t a l  ', '  ( % )  ', '< i > E v a n d r o m y i a   w a l k e r i < / i >', '4 4', '4 0', '1', '1 5 6', '2 9 6', '1', '5 1', '1', '2', '5 9 2', '6 8 . 8 4', '< i > E v a n d r o m y i a   i n f r a s p i n o s a < / i >', '4 4', '0', '3', '8 2', '1', '0', '0', '0', '0', '1 3 0', '1 5 . 1 2', '< i > N y s s o m y i a   a n t u n e s i < s u p > a < / s u p > < / i >', '1 1', '3', '3', '2 0', '3', '0', '1', '0', '0', '4 1', '4 . 7 7', '< i > M i c r o p y g o m y i a   r o r o t a e n s i s < / i >', '2 0', '1', '0', '4', '0', '0', '2', '0', '0', '2 7', '3 . 1 4', '< i > S c i o p e m y i a   s o r d e l l i i < / i >', '7', '1', '0', '1 3', '2', '0', '2', '0', '0', '2 5', '2 . 9 1', '< i > B i c h r o m o m y i a   f l a v 

https://deepayan137.github.io/blog/markdown/2020/08/29/building-ocr.html#the-ctc-loss

In [7]:
# From https://deepayan137.github.io/blog/markdown/2020/08/29/building-ocr.html#the-ctc-loss
class CustomCTCLoss(torch.nn.Module):
    # T x B x H => Softmax on dimension 2
    def __init__(self, dim=2):
        super().__init__()
        self.dim = dim
        self.ctc_loss = torch.nn.CTCLoss(reduction='mean', zero_infinity=True)

    def forward(self, logits, labels,
            prediction_sizes, target_sizes):
        EPS = 1e-7
        loss = self.ctc_loss(logits, labels, prediction_sizes, target_sizes)
        loss = self.sanitize(loss)
        return self.debug(loss, logits, labels, prediction_sizes, target_sizes)
    
    def sanitize(self, loss):
        EPS = 1e-7
        if abs(loss.item() - float('inf')) < EPS:
            return torch.zeros_like(loss)
        if math.isnan(loss.item()):
            return torch.zeros_like(loss)
        return loss

    def debug(self, loss, logits, labels,
            prediction_sizes, target_sizes):
        if math.isnan(loss.item()):
            print("Loss:", loss)
            print("logits:", logits)
            print("labels:", labels)
            print("prediction_sizes:", prediction_sizes)
            print("target_sizes:", target_sizes)
            raise Exception("NaN loss obtained. But why?")
        return loss

In [8]:
# Load alpabet from file
def load_alphabet(file_path):
    with open(file_path, 'r') as f:
        alphabet = f.read().splitlines()
    return alphabet

alphabet = load_alphabet(char_alphabet_file_path)

In [9]:
# Setup environment for training
import numpy as np
from collections import OrderedDict
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.nn.utils.clip_grad import clip_grad_norm_
from tqdm import *
from cnn_lstm_ocr import CNNLSTM_OCR

params = {
    "input_dim": 256,
    "hidden_dim": 256,
    "output_dim": 1,
    "input_planes": 1,
    "planes": 1,
    "schedule": False,
    'image_height':32,
    'number_channels':1,
    'number_hidden_layers':256,
    'len_alphabet':len(alphabet),
    'learning_rate':0.001,
    'epochs':4,
    'batch_size':10,
    'model_dir':'model_history',
    'log_dir':'logs',
    'resume':False,
    'cuda':False,
    'schedule':False    
}


model = CNNLSTM_OCR(params)
criterion = CustomCTCLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])
scheduler = CosineAnnealingLR(optimizer, T_max=params['epochs'])
batch_size = params['batch_size']
count = 1
epochs = params['epochs']
cuda = params['cuda']
print("Params:", params)

Params: {'input_dim': 256, 'hidden_dim': 256, 'output_dim': 1, 'input_planes': 1, 'planes': 1, 'schedule': False, 'image_height': 32, 'number_channels': 1, 'number_hidden_layers': 256, 'len_alphabet': 113, 'learning_rate': 0.001, 'epochs': 4, 'batch_size': 10, 'model_dir': 'model_history', 'log_dir': 'logs', 'resume': False, 'cuda': False}


In [10]:
import torch
import torch.nn as nn
from torchvision import transforms
from torch.optim.lr_scheduler import CosineAnnealingLR
from tqdm import tqdm
from custom_ocr_cnn_lstm.dataset_cropped import OCRDatasetCropped

# Define a method for training the OCR model
def train_ocr_model(model, train_loader, optimizer, criterion, device, epoch, epochs):
    model.train()
    total_loss = 0.0

    progress_bar = tqdm(train_loader, desc='Epoch: [%d]/[%d] Training'%(epoch, 
                epochs), leave=True)

    for labels, images in enumerate(progress_bar):
        print("Images:", images.shape)
        print("Labels:", labels.shape)
        optimizer.zero_grad()
        
        # Move data to the specified device (CPU or GPU)
        images, labels = images.to(device), labels.to(device)
        
        # Forward pass
        outputs = model(images)
        
        # Calculate the loss
        loss = criterion(outputs, labels)
        
        # Backpropagation
        loss.backward()
        clip_grad_norm_(model.parameters(), max_norm=5)  # Clip gradients to prevent exploding gradients
        
        optimizer.step()
        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_loader)
    return avg_loss

# Define a method for evaluating the OCR model on the validation dataset
def evaluate_ocr_model(model, val_loader, criterion, device):
    model.eval()
    total_loss = 0.0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Validation"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
    
    avg_loss = total_loss / len(val_loader)
    return avg_loss

# Define a method for OCR inference
def ocr_inference(model, image, transform, device):
    model.eval()
    with torch.no_grad():
        # Preprocess the input image using the same transform as during training
        image = transform(image).unsqueeze(0).to(device)
        outputs = model(image)
        
        # Perform any necessary post-processing on the model's outputs to get the OCR result
        # You may need to implement this part based on your specific OCR task
    
    return ocr_result


# Set device (CPU or GPU) for training
device = torch.device("cuda" if params['cuda'] and torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
for epoch in range(params['epochs']):
    print(f"Epoch {epoch + 1}/{params['epochs']}")
    
    # Train the model
    train_loss = train_ocr_model(model, train_loader, optimizer, criterion, device, epoch, params['epochs'])
    print(f"Train Loss: {train_loss:.4f}")
    
    # Validate the model
    val_loss = evaluate_ocr_model(model, val_loader, criterion, device)
    print(f"Validation Loss: {val_loss:.4f}")

# After training, you can use the trained model for inference on new images
# Example usage for OCR inference:
sample_image, sample_label = train_dataset[0]  # Load a sample image and label
ocr_result = ocr_inference(model, sample_image, transform, device)
print("OCR Result:", ocr_result)


Epoch 1/4


Epoch: [0]/[4] Training:   0%|          | 0/685 [00:00<?, ?it/s]


RuntimeError: each element in list of batch should be of equal size

In [ ]:
# Iterate over train dataset
last_image = None
total_diff = 0

last_label = None
diff_label_len = 0
for images, labels in train_dataset:
    # print("Images:", images.shape)
    # Compute shape diff to previous image
    if last_image is not None:
        total_diff += images.shape[1] - last_image.shape[1]
    last_image = images
    # Compute label diff to previous label
    if diff_label_len is not None:
        diff_label_len += len(labels) - diff_label_len
    last_label = labels

print("Total diff:", total_diff)
print("Diff label len:", diff_label_len)

Total diff: 0
Diff label len: 35
